# LSTM com Embeddings (Feat. Machado de Assis)

In [126]:
import collections
from typing import List, Set, Dict, Tuple, Generator

import numpy as np
import pandas as pd
import scipy
import tensorflow as tf

CAMINHO_MODELO = "modelo.json"
CAMINHO_DICIONARIO = "dicionario.json"
CAMINHO_DICIONARIO_INDICES = "dicionario_indices.json"
EMBEDDING_UNITS = 256
LSTM_UNITS = 256

In [79]:
def criar_dicionario(palavras: List[str], minimo_palavras: int) -> (Dict[str, int], Dict[int, str], Dict[str, int]):
    dicionario_freq = collections.Counter(palavras)
    palavras_no_dicionario = [palavra for palavra, qtd in dicionario_freq.items() if qtd > minimo_palavras]
    dicionario = {palavra: indice for (indice, palavra) in enumerate(palavras_no_dicionario)}
    dicionario["UNK"] = len(dicionario)
    dicionario_indices = dict(zip(dicionario.values(), dicionario.keys()))
    return dicionario, dicionario_indices, dicionario_freq
df_obras = pd.read_csv("./obras_machado_de_assis.csv")
df_obras = df_obras[df_obras["categoria"] != "tradução"]

minimo_palavras_frase = 2
dataset = pd.Series(np.concatenate(df_obras["texto"].str.replace("\n+", " ").str.replace("\.+", ".").str.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s").values))
dataset = dataset.str.replace("[^\w\s\d]", "").str.strip().str.lower()
dataset = dataset[dataset.apply(lambda row: len(row.split())) > minimo_palavras_frase].reset_index(drop=True)
palavras = np.concatenate(dataset.apply(lambda row: row.split()))

minimo_palavras_dicionario = 2
dicionario, dicionario_indices, dicionario_freq = criar_dicionario(palavras, minimo_palavras_dicionario)
dataset = dataset.apply(lambda row: [dicionario.get(palavra, len(dicionario)-1) for palavra in row.split()])
tamanho_dicionario = len(dicionario)
tamanho_dataset = len(dataset)

In [80]:
print("Frases no dataset: {0}\nTotal de palavras: {1}\nPalavras no dicionário: {2}\nMédia de palavras por frase: {3}".format(len(dataset), len(palavras), len(dicionario), round(len(palavras)/len(dataset), 1)))

Frases no dataset: 98366
Total de palavras: 1698151
Palavras no dicionário: 28406
Média de palavras por frase: 17.3


In [81]:
dataset.head()

0                                            [0, 1, 2]
1                                         [3, 4, 5, 6]
2              [7, 8, 9, 10, 2, 11, 12, 13, 12, 14, 2]
3    [3, 15, 16, 17, 15, 18, 19, 20, 2, 21, 22, 23,...
4    [49, 50, 51, 52, 53, 54, 35, 29, 22, 55, 2, 56...
dtype: object

In [131]:
def criar_modelo(dim_entrada, dim_embedding, dim_lstm):
    entrada = tf.keras.layers.Input((None,), name="entrada")
    camada_embedding = tf.keras.layers.Embedding(dim_entrada, dim_embedding, name='embedding')
    camada_contexto = camada_embedding(entrada)
    camada_lstm = tf.keras.layers.LSTM(dim_lstm, name="lstm")(camada_contexto)
    camada_saida = tf.keras.layers.Dense(1,name="saida", activation="sigmoid")(camada_lstm)
    
    modelo = tf.keras.models.Model(inputs=entrada, outputs=camada_saida)
    modelo.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc"])
    
    return modelo

    
modelo = criar_modelo(dim_entrada=tamanho_dicionario, dim_embedding=EMBEDDING_UNITS, dim_lstm=LSTM_UNITS)
modelo.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
entrada (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 256)         7271936   
_________________________________________________________________
lstm (LSTM)                  (None, 256)               525312    
_________________________________________________________________
saida (Dense)                (None, 1)                 257       
Total params: 7,797,505
Trainable params: 7,797,505
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def criar_gerador(dataset, tamanho_dicionario):
    while 42:
        batchX = []
        batchY = []
        for idx, d in enumerate(dataset):
            positivo = np.array(d)
            negativo = np.array(d)
            batchX.append(positivo)
            batchY.append(1)
            np.random.shuffle(negativo)
            batchX.append(negativo)
            batchY.append(0)
        ordem = np.arange(len(batchY))
        np.random.shuffle(ordem)
        batchX = np.array(batchX)[ordem]
        batchY = np.array(batchY)[ordem].reshape((-1, 1))
        for x, y in zip(batchX, batchY):
            yield (x.reshape(1, -1), y)


gerador = criar_gerador(dataset, tamanho_dicionario)
steps = 2*(tamanho_dataset-1)
modelo.fit_generator(gerador, epochs=1, steps_per_epoch=steps, validation_steps=round(steps*0.1))

 37655/196730 [====>.........................] - ETA: 5:12:52 - loss: 0.3069 - acc: 0.8556

In [115]:
gerador = criar_gerador(dataset, tamanho_dicionario)
modelo.evaluate_generator(gerador, steps=1000)

[0.14643467954319203, 0.941]

In [120]:
frase = "dinheiro é uma coisa".lower()
probs = []
for palavra, indice in dicionario.items():
    frase2 = frase + " " + palavra
    vetor = np.array([[dicionario.get(palavra, tamanho_dicionario-1) for palavra in frase2.split()]])
    probs.append((palavra, modelo.predict(vetor)))
print(probs[:10])

[('poesias', array([[0.65413696]], dtype=float32)), ('de', array([[0.0027519]], dtype=float32)), ('a', array([[0.02708883]], dtype=float32)), ('zaluar', array([[0.60680383]], dtype=float32)), ('garnier', array([[0.5793376]], dtype=float32)), ('editor', array([[0.85829216]], dtype=float32)), ('1863', array([[0.81716174]], dtype=float32)), ('dois', array([[0.52060324]], dtype=float32)), ('motivos', array([[0.4067607]], dtype=float32)), ('me', array([[0.00091455]], dtype=float32))]


In [121]:
sorted(probs, key=lambda x: -x[1][0])[:20]

[('comum', array([[0.9707183]], dtype=float32)),
 ('antiga', array([[0.9638997]], dtype=float32)),
 ('extraordinária', array([[0.9633721]], dtype=float32)),
 ('diferente', array([[0.96310014]], dtype=float32)),
 ('sério', array([[0.96287674]], dtype=float32)),
 ('deveras', array([[0.96188235]], dtype=float32)),
 ('dela', array([[0.9614694]], dtype=float32)),
 ('humano', array([[0.96115816]], dtype=float32)),
 ('embora', array([[0.96082306]], dtype=float32)),
 ('morto', array([[0.9606224]], dtype=float32)),
 ('bonitas', array([[0.9603592]], dtype=float32)),
 ('dalma', array([[0.96016157]], dtype=float32)),
 ('direta', array([[0.95954347]], dtype=float32)),
 ('calado', array([[0.9589443]], dtype=float32)),
 ('público', array([[0.95864725]], dtype=float32)),
 ('moral', array([[0.958546]], dtype=float32)),
 ('sozinha', array([[0.9582255]], dtype=float32)),
 ('pura', array([[0.95819914]], dtype=float32)),
 ('familiar', array([[0.9581987]], dtype=float32)),
 ('cedo', array([[0.9580307]], dty

In [122]:
sorted(probs, key=lambda x: x[1][0])[:20]

[('me', array([[0.00091455]], dtype=float32)),
 ('lhe', array([[0.00095359]], dtype=float32)),
 ('no', array([[0.00108303]], dtype=float32)),
 ('te', array([[0.00135581]], dtype=float32)),
 ('da', array([[0.00139425]], dtype=float32)),
 ('ao', array([[0.0016248]], dtype=float32)),
 ('dos', array([[0.00175668]], dtype=float32)),
 ('na', array([[0.00187941]], dtype=float32)),
 ('pelas', array([[0.00222989]], dtype=float32)),
 ('e', array([[0.0027326]], dtype=float32)),
 ('de', array([[0.0027519]], dtype=float32)),
 ('do', array([[0.00278761]], dtype=float32)),
 ('dessa', array([[0.0030639]], dtype=float32)),
 ('pelos', array([[0.00308376]], dtype=float32)),
 ('aos', array([[0.00312166]], dtype=float32)),
 ('o', array([[0.00331012]], dtype=float32)),
 ('à', array([[0.00375244]], dtype=float32)),
 ('pela', array([[0.00399444]], dtype=float32)),
 ('num', array([[0.00421807]], dtype=float32)),
 ('neste', array([[0.00437084]], dtype=float32))]

In [ ]:
for epoch in range(10):
    hidden = modelo.reset_states()
    for (batch, (input, target)) in enumerate(ds):
        with tf.GradientTape() as tape:
            predictions, hidden = modelo(input, hidden)
            target = tf.reshape(target, (-1,))
            loss = loss_function(target, predictions)
            grads = tape.gradient(loss, modelo.variables)
            optimizer.apply_gradients(zip(grads, modelo.variables))
            if batch % 1000 == 0:
                print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, loss))

In [ ]:
start_string = "Eduardo viajou para o que"

input_eval = [dicionario.get(string) for string in start_string.lower().split()]
input_eval = tf.expand_dims(input_eval, 0)

text_generated = ''

hidden = [tf.zeros((1, LSTM_UNITS))]

predictions, hidden = modelo(input_eval, hidden)

predicted_id = tf.argmax(predictions[-1]).numpy()

text_generated += " " + dicionario_indices[predicted_id]

print(start_string + text_generated)

In [ ]:
def criar_gerador(dataset, dicionario):
    for frase in dataset:
        
#         yield (np.array([frase[:-1]]), np.array([frase[-1]]))
        
next(criar_gerador(dataset, dicionario))

# gerador = criar_gerador(dataset, dicionario, modelo_embedding)
# modelo.fit_generator(gerador, epochs=1, steps_per_epoch=len(dataset)-1)

In [ ]:
embeddings = modelo_embedding.predict(np.arange(len(dicionario)))
embeddings

In [ ]:
frase = "a carne mais barata do mercado é a carne tendo".lower()
entrada = [dicionario.get(palavra, len(dicionario)-1) for palavra in frase.split()]
vec = modelo.predict(entrada)
[dicionario_indices[indice] for indice in distancia_coseno(embeddings, vec).argmin(axis=0)]

In [ ]:
[dicionario_indices[indice] for indice in (np.dot(vec, embeddings.T)/vec.sum(axis=1).reshape((-1, 1))/embeddings.sum(axis=1)).argmin(axis=1)]

In [ ]:
def distancia_coseno(embeddings, vetores):
    return (np.dot(embeddings, vec.T)/(vec**2).sum(axis=1)**0.5)/((embeddings**2).sum(axis=1)**0.5).reshape((-1, 1))
# np.dot(vec, embeddings.T)/vec.sum(axis=1).reshape((-1, 1))/embeddings.sum(axis=1)
x
#  (vec**2).sum(axis=1)/embeddings.sum(axis=1).reshape((-1, 1))) 

In [ ]:
# np.argmax((embeddings - vec).sum(axis=0))
np.array([embeddings - v for v in vec]).sum(axis=2).argmax(axis=1)

In [ ]:
dicionario_indices[0]

In [ ]:
class Model(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, units, batch_size):
        super(Model, self).__init__()
        self.units = units
        self.batch_size = batch_size
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_activation='sigmoid',
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size) 
    def call(self, inputs, hidden):
        inputs = self.embedding(inputs)
        output, states = self.gru(inputs, initial_state=hidden)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, states

BATCH_SIZE = 100
modelo = Model(len(dicionario), EMBEDDINGS, LSTM_UNITS, BATCH_SIZE)
# modelo.build((10,))
# modelo.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["acc"])
# modelo, modelo_embedding = criar_modelo(len(dicionario), EMBEDDINGS)
# modelo_embedding.summary()
optimizer = tf.train.AdamOptimizer()
def loss_function(labels, logits):
    return tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

X, Y = [], []
for d in dataset:
    for i in range(1, len(d)):
        X.append(d[i-1])
        Y.append(d[i])

X, Y = np.array(X).reshape((-1, 1)), np.array(Y).reshape(-1, 1)
ds = tf.data.Dataset.from_tensor_slices((X, Y))
ds = ds.batch(BATCH_SIZE, drop_remainder=True)


for epoch in range(1):
    hidden = modelo.reset_states()
    for (batch, (input, target)) in enumerate(ds):
        with tf.GradientTape() as tape:
            predictions, hidden = modelo(input, hidden)
            target = tf.reshape(target, (-1,))
            loss = loss_function(target, predictions)
            grads = tape.gradient(loss, modelo.variables)
            optimizer.apply_gradients(zip(grads, modelo.variables))
            if batch % 100 == 0:
                print('Epoch {} Batch {} Loss{:.4f}'.format(epoch + 1, batch, loss))

In [ ]:
sorted(get_preds("não"), key=lambda x: x[1])

In [ ]:
get_preds("quatro")